In [2]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import time

In [3]:
# create path and open browser window
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# establish url
url = 'https://mars.nasa.gov/news/'

# visit site
browser.visit(url)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/lbates/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [3]:
time.sleep(1)
# create soup object
soup = BeautifulSoup(browser.html,'html.parser')

In [4]:
# find title for the latest one, which is the one in the first box
news_title = soup.find_all('div',class_='content_title')
news_title = news_title[1].text
news_title

'NASA Ingenuity Mars Helicopter Prepares for First Flight'

In [5]:
# pulling the text from the paragraph
news_p = soup.find_all('div', class_= 'article_teaser_body')
news_p = news_p[0].text
news_p

'Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  '

In [6]:
# establish url
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# visit site
browser.visit(url)

# grab page html
jpl_html = browser.html

# create soup object
soup = BeautifulSoup(jpl_html,'html.parser')

In [7]:
#Find the image at the top and click 
target = 'a[class="group  cursor-pointer block"]'
browser.find_by_tag(target).click()

time.sleep(1)


In [8]:

# grab page html
target_html = browser.html

# create soup object
soup = BeautifulSoup(target_html,'html.parser')

#find all anchors and loop through to find href by using get_text for JPG
anchors = soup.find_all('a')
for a in anchors:
    if 'JPG' in a.get_text():
        featured_image_url=a['href']
    
#print(anchors)
#featured_image_url = image[0]['href']
featured_image_url

'https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA23810.jpg'

In [5]:
# establish url
facts_url = 'https://space-facts.com/mars/'

# visit site
browser.visit(facts_url)

#pull the table from the site
tables =pd.read_html('https://space-facts.com/mars/')
#pull the specific table for just the Mars data, it's the first table
mars_tables = tables[0]
#rename columns
mars_tables.columns = ['Fact','Value']
#convert dataframe back to html
mars_tables_html = mars_tables.to_html()
mars_tables_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Fact</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>

In [10]:
# establish url
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# visit site
browser.visit(hemi_url)

# grab page html
hemi_html = browser.html

# create soup object
soup = BeautifulSoup(hemi_html,'html.parser')

In [11]:
#Find the all the titles to use for the click function 
title_list = soup.find_all('div', class_='description')

hemisphere_image_urls = []
#loop through the title_list and pull title and imagine url
for t in title_list:
    title = t.h3.text
    print(title)
    browser.find_by_text(title).click()

    # grab page html
    title_html = browser.html

    # create soup object
    soup = BeautifulSoup(title_html,'html.parser')

    image = soup.find_all('div', class_='downloads')
    image_url = image[0].li.a['href']
    
    #create mini-dictionary
    mini_dictionary = {'title': title, 'img_url':image_url}
    hemisphere_image_urls.append(mini_dictionary)
    
    # click back button
    browser.back()
    
hemisphere_image_urls



Cerberus Hemisphere Enhanced
Schiaparelli Hemisphere Enhanced
Syrtis Major Hemisphere Enhanced
Valles Marineris Hemisphere Enhanced


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [6]:
#close the browser
browser.quit()